# Pickle NSL-KDD

This notebooks intended use is to load the CSV data into a Pandas dataframe, normalize and scale the data, then write the DataFrame into a pickle to save these steps for every ML framework run.  
The output are four pickle files: kdd_train_data, kdd_train_labels, kdd_test_data and kdd_test_labels.  
These pickles can be restored as dataframes by calling [pandas.read_pickle()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_pickle.html).

In [1]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
pd.set_option('display.max_columns', None)

Using TensorFlow backend.


## Data Loading and Prep

Fist of all, load the header, so Pandas is able to identify the columns correctly.  
As the header info for the last two columns is missing, I am adding these by hand to the DataFrame.  
They are the type of traffic, *label*, and the difficulty of detection, *difficulty_level*.  
These columns are removed from the data-pickled but for completeness' sake, I'm including these.

In [2]:
print('Loading header info from "Field Names.csv"')
header_col = pd.read_csv(os.path.join('NSL_KDD', 'Field Names.csv'), header=None)
header_col = header_col.append(pd.DataFrame([['label','symbolic'],['difficulty_level','continuous']]))

header_names = header_col[0].values
print('loaded {} header names: {}'.format(len(header_names), header_names))

Loading header info from "Field Names.csv"
loaded 43 header names: ['duration' 'protocol_type' 'service' 'flag' 'src_bytes' 'dst_bytes'
 'land' 'wrong_fragment' 'urgent' 'hot' 'num_failed_logins' 'logged_in'
 'num_compromised' 'root_shell' 'su_attempted' 'num_root'
 'num_file_creations' 'num_shells' 'num_access_files' 'num_outbound_cmds'
 'is_host_login' 'is_guest_login' 'count' 'srv_count' 'serror_rate'
 'srv_serror_rate' 'rerror_rate' 'srv_rerror_rate' 'same_srv_rate'
 'diff_srv_rate' 'srv_diff_host_rate' 'dst_host_count'
 'dst_host_srv_count' 'dst_host_same_srv_rate' 'dst_host_diff_srv_rate'
 'dst_host_same_src_port_rate' 'dst_host_srv_diff_host_rate'
 'dst_host_serror_rate' 'dst_host_srv_serror_rate' 'dst_host_rerror_rate'
 'dst_host_srv_rerror_rate' 'label' 'difficulty_level']


### Training Set

We're using "KDDTrain+" specifically, as this is the full training data set.

In [3]:
ftrain = os.path.join('NSL_KDD','KDDTrain+.csv')
kdd_train_data = pd.read_csv(ftrain, header=None, names=header_names)

# split off labels
kdd_train_labels = kdd_train_data.filter(['label','difficulty_level'])

# ...and drop them as well as the difficulty level
kdd_train_data.drop(['label', 'difficulty_level'],axis=1,inplace=True)

kdd_train_data.tail()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
125968,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.0,255,25,0.10,0.06,0.00,0.0,1.00,1.0,0.00,0.0
125969,8,udp,private,SF,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.0,255,244,0.96,0.01,0.01,0.0,0.00,0.0,0.00,0.0
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.0,255,30,0.12,0.06,0.00,0.0,0.72,0.0,0.01,0.0
125971,0,tcp,klogin,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.0,255,8,0.03,0.05,0.00,0.0,1.00,1.0,0.00,0.0
125972,0,tcp,ftp_data,SF,151,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.0,255,77,0.30,0.03,0.30,0.0,0.00,0.0,0.00,0.0


### Test Set

As with the Training set, we're using the full "KDDTest+" file.

In [4]:
ftest = os.path.join('NSL_KDD','KDDTest+.csv')
kdd_test_data = pd.read_csv(ftest, header=None, names=header_names)

# split off labels..
kdd_test_labels = kdd_test_data.filter(['label','difficulty_level'])

# ...and drop them as well as the difficulty level
kdd_test_data.drop(['label', 'difficulty_level'],axis=1,inplace=True)

kdd_test_data.tail()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
22538,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.0,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22539,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.0,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22540,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.0,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22541,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.0,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00
22542,0,tcp,sunrpc,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,10,0.0,0.0,1.0,1.0,0.25,1.0,1.00,255,21,0.08,0.03,0.00,0.00,0.00,0.0,0.44,1.00


## Attack Label Encoding

As the labels are still plaintext, they need to be converted to simple integer representations.

In [5]:
all_labels = pd.concat([kdd_train_labels, kdd_test_labels])

In [6]:
label_tokenizer = Tokenizer(num_words=len(all_labels['label'].unique())+1, filters='')
label_tokenizer.fit_on_texts(all_labels['label'].unique())

### Train Data

In [7]:
encoded_train_labels = pd.DataFrame(columns=['label_encoded'],dtype=np.int8,data=label_tokenizer.texts_to_sequences(kdd_train_labels['label']))

In [8]:
kdd_train_labels = pd.concat([kdd_train_labels, encoded_train_labels], axis=1)
kdd_train_labels.head()

,label,difficulty_level,label_encoded
0,normal,20,1
1,normal,15,1
2,neptune,19,2
3,normal,21,1
4,normal,21,1


### Test Data

In [9]:
encoded_test_labels = pd.DataFrame(columns=['label_encoded'],dtype=np.int8,data=label_tokenizer.texts_to_sequences(kdd_test_labels['label']))
kdd_test_labels = pd.concat([kdd_test_labels, encoded_test_labels], axis=1)
kdd_test_labels.head()

,label,difficulty_level,label_encoded
0,neptune,21,2
1,neptune,21,2
2,normal,21,1
3,saint,15,24
4,mscan,11,25


## Remaining Column Encoding

Besides the label, there are three more columns that need to be translated from text to integer data: __protocol_type__, __service__ and __flag__.  
As these are done the same way, I define a function that fits a tokenizer, transforms the texts and appends the encoded column.  
The plaintext columns are dropped, as this is needed for normalization. The regarding class indexes are written as JSON to *dataset_columnName_wordindex.json* along with the pickles.

In [10]:
# create one big dataframe for training the encoders
all_data = pd.concat([kdd_train_data, kdd_test_data])

In [11]:
import json

def encode_column_to_int(dataframe, column, filename):
    # instantiate and fit a tokenizer on exactly the number of values present in the column
    col_encoder = Tokenizer(num_words=len(all_data[column].unique())+1, filters='')
    col_encoder.fit_on_texts(all_data[column].unique())
    
    
    print('Writing encoder data to file {}: {}'.format(filename, col_encoder.word_index))
    with open(filename, 'w') as outfile:
        json.dump(col_encoder.word_index, outfile)

    # then transform the column in question
    encoded_col = pd.DataFrame(columns=[column],dtype=np.int8,data=col_encoder.texts_to_sequences(dataframe[column]))
    
    #drop the source column
    dataframe.drop([column],axis=1,inplace=True)

    # and append the encoded one to the original dataframe
    dataframe = pd.concat([dataframe, encoded_col], axis=1)
    return dataframe

In [12]:
kdd_train_data = encode_column_to_int(kdd_train_data, 'protocol_type', os.path.join('NSL_KDD','kdd_train_data_protocol_type_wordindex.json'))
kdd_test_data = encode_column_to_int(kdd_test_data, 'protocol_type', os.path.join('NSL_KDD','kdd_test_data_protocol_type_wordindex.json'))

Writing encoder data to file NSL_KDD/kdd_train_data_protocol_type_wordindex.json: {'tcp': 1, 'udp': 2, 'icmp': 3}
Writing encoder data to file NSL_KDD/kdd_test_data_protocol_type_wordindex.json: {'tcp': 1, 'udp': 2, 'icmp': 3}


In [13]:
kdd_train_data = encode_column_to_int(kdd_train_data, 'service', os.path.join('NSL_KDD','kdd_train_data_service_wordindex.json'))
kdd_test_data = encode_column_to_int(kdd_test_data, 'service', os.path.join('NSL_KDD','kdd_test_data_service_wordindex.json'))

Writing encoder data to file NSL_KDD/kdd_train_data_service_wordindex.json: {'ftp_data': 1, 'other': 2, 'private': 3, 'http': 4, 'remote_job': 5, 'name': 6, 'netbios_ns': 7, 'eco_i': 8, 'mtp': 9, 'telnet': 10, 'finger': 11, 'domain_u': 12, 'supdup': 13, 'uucp_path': 14, 'z39_50': 15, 'smtp': 16, 'csnet_ns': 17, 'uucp': 18, 'netbios_dgm': 19, 'urp_i': 20, 'auth': 21, 'domain': 22, 'ftp': 23, 'bgp': 24, 'ldap': 25, 'ecr_i': 26, 'gopher': 27, 'vmnet': 28, 'systat': 29, 'http_443': 30, 'efs': 31, 'whois': 32, 'imap4': 33, 'iso_tsap': 34, 'echo': 35, 'klogin': 36, 'link': 37, 'sunrpc': 38, 'login': 39, 'kshell': 40, 'sql_net': 41, 'time': 42, 'hostnames': 43, 'exec': 44, 'ntp_u': 45, 'discard': 46, 'nntp': 47, 'courier': 48, 'ctf': 49, 'ssh': 50, 'daytime': 51, 'shell': 52, 'netstat': 53, 'pop_3': 54, 'nnsp': 55, 'irc': 56, 'pop_2': 57, 'printer': 58, 'tim_i': 59, 'pm_dump': 60, 'red_i': 61, 'netbios_ssn': 62, 'rje': 63, 'x11': 64, 'urh_i': 65, 'http_8001': 66, 'aol': 67, 'http_2784': 68, '

In [14]:
kdd_train_data = encode_column_to_int(kdd_train_data, 'flag', os.path.join('NSL_KDD','kdd_train_data_flag_wordindex.json'))
kdd_test_data = encode_column_to_int(kdd_test_data, 'flag', os.path.join('NSL_KDD','kdd_test_data_flag_wordindex.json'))

Writing encoder data to file NSL_KDD/kdd_train_data_flag_wordindex.json: {'sf': 1, 's0': 2, 'rej': 3, 'rstr': 4, 'sh': 5, 'rsto': 6, 's1': 7, 'rstos0': 8, 's3': 9, 's2': 10, 'oth': 11}
Writing encoder data to file NSL_KDD/kdd_test_data_flag_wordindex.json: {'sf': 1, 's0': 2, 'rej': 3, 'rstr': 4, 'sh': 5, 'rsto': 6, 's1': 7, 'rstos0': 8, 's3': 9, 's2': 10, 'oth': 11}


In [15]:
kdd_test_data.tail()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type,service,flag
22538,0,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.0,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,1,16,1
22539,0,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.0,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,1,4,1
22540,0,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.0,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,1,4,1
22541,0,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.0,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,2,12,1
22542,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,10,0.0,0.0,1.0,1.0,0.25,1.0,1.00,255,21,0.08,0.03,0.00,0.00,0.00,0.0,0.44,1.00,1,38,3


## Feature Standardization

As the scaler converts the DataFrames to numpy arrays, save the header info to recreate a DataFrame afterwards.

In [16]:
train_data_header = list(kdd_train_data.columns.values)
test_data_header = list(kdd_test_data.columns.values)

As many ML implementations behave badly if confronted with non-scaled inputs, we go ahead and transform all features to center, then scale it.

In [17]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(kdd_train_data) # fit the scaler on the training data

# transform both samples without any refitting
kdd_train_data = scaler.transform(kdd_train_data)
kdd_test_data = scaler.transform(kdd_test_data)

In [18]:
kdd_train_data = pd.DataFrame(columns=train_data_header, data=kdd_train_data)
kdd_test_data = pd.DataFrame(columns=test_data_header, data=kdd_test_data)

In [19]:
kdd_test_data.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type,service,flag
0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.448141,0.019569,0.0,0.00,1.0,1.0,0.04,0.06,0.00,1.000000,0.039216,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,0.0,0.028986,0.2
1,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.266145,0.001957,0.0,0.00,1.0,1.0,0.01,0.06,0.00,1.000000,0.003922,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,0.0,0.028986,0.2
2,0.000047,9.408217e-06,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.525490,0.337255,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,0.0,0.000000,0.0
3,0.000000,1.449313e-08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.127202,0.0,0.00,0.0,0.0,1.00,0.00,1.00,0.011765,0.223529,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,1.0,0.101449,0.0
4,0.000023,0.000000e+00,1.145093e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.015656,0.0,0.12,1.0,0.5,1.00,0.00,0.75,0.113725,0.337255,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,0.0,0.130435,0.5


## Serialization

So at this point, we have training and test sets with data and labels. The data parts are encoded and scaled, the encoded indizes are written away as json files.  
It would be nice if this data could be used for future runs, right? Right!  
That's why we serialize each dataframe into a python binary pickle on it's own (which is a feature directly supported by [Pandas](https://pandas.pydata.org/pandas-docs/stable/api.html#id12) - nice, eh?)

In [20]:
def write_to_pickle(dataframe, filename):
    dataframe.to_pickle(os.path.join('NSL_KDD', filename+'.pkl'))

In [21]:
write_to_pickle(kdd_train_data, 'kdd_train_data')
write_to_pickle(kdd_test_data, 'kdd_test_data')
write_to_pickle(kdd_train_labels, 'kdd_train_labels')
write_to_pickle(kdd_test_labels, 'kdd_test_labels')

These can be loaded into a Pandas DataFrame like this:  
`someDataFrame = pd.read_pickle("./dummy.pkl")`